# Wczytanie potrzebnych bibliotek


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import requests

# Wczytanie danych z naszego API oraz konwersja na PandasDataFrame

In [3]:
df=requests.get("https://api.apispreadsheets.com/api/dataset/forest-fires/").json()
from pandas import json_normalize 
df = json_normalize(df, 'data')


# Sprawdźmy z jakich kolumn składa się nasz DataFrame

In [ ]:
df.head(4)

In [ ]:
df.info()

Jak możemy zauważyć, wszystkie nasze kolumny składają się z wartości nie wybrakowanych (non-null) oraz każda z kolumn ma ujednolicony typ danych 

# Sprawdzenie rozkładów poszczególnych wartości w kolumnach

In [ ]:
df.describe()

In [ ]:
df.hist(figsize=(18, 12), bins=30)
plt.show()

In [ ]:
plt.plot(df['area'], 'o')
plt.title('Powierzchnia która spłonęła w kolejnych wystąpieniach')
plt.xlabel('wystąpienia')
plt.ylabel('area')
plt.show()

In [ ]:
plt.plot(df['rain'], 'o')
plt.title('Powierzchnia która spłonęła w kolejnych wystąpieniach')
plt.xlabel('wystąpienia')
plt.ylabel('area')
plt.show()

In [ ]:
box = sns.boxplot(df['area'])
box.set_title('Area')

Z naszych wizualizacji możemy zauważyć między innymi to, że:
* znacząca część lasów nie płonęła
* w lasach z naszej bazy danych rzadko kiedy padał deszcz


Zauważmy, że posiadamy koordynaty naszych lasów. Sprawdźmy więc, czy w naszych danych nie znajdziemy miejsca, w którym pożary lasów występują znacząco częściej niż w pozostałych

In [ ]:
df_region_mean = df.groupby(["X","Y"])
df_region_mean = df_region_mean.mean().reset_index()
df_coordinate = df_region_mean.pivot("X", "Y", "area")
plt.figure(figsize=(7, 7))
sns.heatmap(df_coordinate, cmap="Reds", annot=True, fmt=".0f")
plt.title("Średnia powierzchnia części lasu o koordynatach X i Y która spłonęła")
plt.show()



Jak możemy zauważyć, Region X=8 Y=8 charakteryzuje się tym, że średni spalony obszar jest bardzo duży

In [ ]:
df_region_count = df.groupby(["X","Y"]).count().reset_index()
df_region_count
df_region_count = df_region_count.pivot("X", "Y", "area")
plt.figure(figsize=(7, 7))
sns.heatmap(df_region_count, cmap="Reds", annot=True, fmt=".0f")
plt.title("Liczba rekordów dla danego regionu")
plt.show()

In [ ]:
df_88 = df[(df["X"] == 8) & (df["Y"] == 8)]
df_88

Zauważmy także, że dla danego regionu wykonano tylko jeden pomiar. Może to na przykład oznaczać, że dany region spłonął w większości/całości, więc dalsze pomiary nie mają dla niego już sensu

# Sprawdźmy zależności pomiędzy interesującą nas wartością area - części lasu która spłonęła, a pozostałymi zmiennymi.

In [ ]:
cols = df.columns[0:12]
cols
sns.pairplot(df, x_vars=cols[0:4], y_vars ="area", height = 4, aspect = 1)
sns.pairplot(df, x_vars=cols[5:9], y_vars ="area", height = 4, aspect = 1)
sns.pairplot(df, x_vars=cols[10:12], y_vars ="area", height = 4, aspect = 1)
plt.tight_layout()

### Zauważmy, że nie tylko w regionie X=8 oraz Y=8 wystąpił bardzo duży pożar.  Sprawdźmy więc pozostałe miejsca, w których wystąpiły duże pożary (przyjmijmy górne 5%)

In [ ]:
quantile = df.area.quantile(.95)
df_burned_areas = df[df["area"] > quantile]
df_burned_areas

In [ ]:
sns.pairplot(df_burned_areas, x_vars=cols[0:4], y_vars ="area", height = 4, aspect = 1)
sns.pairplot(df_burned_areas, x_vars=cols[5:9], y_vars ="area", height = 4, aspect = 1)
sns.pairplot(df_burned_areas, x_vars=cols[10:12], y_vars ="area", height = 4, aspect = 1)
plt.tight_layout()

In [ ]:
df_burned_areas_count = df_burned_areas.groupby("month").count().reset_index()
df_burned_areas_count = df_burned_areas_count[["month", "X"]]
months = ["feb","apr", "jun", "jul", "aug", "sep", "oct"]
df_burned_areas_count

In [ ]:
df_burned_areas_count["ind"] = ['b', 'e', 'a', 'd', 'c','g', 'f']
df_burned_areas_count = df_burned_areas_count.set_index("ind")
df_burned_areas_count.sort_index(inplace = True)  
sns.pointplot(x=df_burned_areas_count.month, y=df_burned_areas_count.X)

Dzięki naszym zależnością możemy zauważyć, że:
* większość dużych pożarów wystąpiła latem (Montesinho park leży w Portugalii, https://ibb.co/TmXD932)
* w dniu pożaru nie padał deszcz

Sprawdźmy, jak z eksploracją danych radzi sobie pandas profiling

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df)
#profile.to_file(output_file = "report.html")
profile.to_file("output.html")

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Zdecydowanym ograniczeniem narzędzia jest fakt, że produkcja takiego raportu zajęła na moim komputerze średnio 30 sekund. Zauważmy, że nasze dane zawierają tylko 517 obserwacji. Mimo wszystko narzędzie w bardzo prosty sposób pozwala nam zeksplorować nasze dane i z pewnością warto z niego skorzystać w przypadku posiadania niewielkiego zbioru danych